# Advent of Code - 2021

## Daily Data

The input data for each day "e.g. day 5" should be downloaded into the data subdirectory for the corresponding day (e.g. data/day_5/input.txt). This will allow use of the `get_daily_data` generator function.

## Support Functions

In [1]:
import os
def get_daily_data(day, sep=None):
    daily_filename = "data/day_{}/input.txt".format(day)
    if not os.path.exists(daily_filename):
        try:
            os.rename("Users/mikes/Downloads/input.txt", daily_filename)
        except:
            raise Exception("Could not find daily input file.")
    with open(daily_filename, 'r') as fin:
        for line in fin.readlines():
            if sep is None:
                yield line
            else:
                yield line.split(sep)

## Day 3

In [9]:
"helloworld"[::-1]

'dlrowolleh'

## Day 2

In [2]:
horizontal_loc = 0
depth_loc = 0
for direction, value in get_daily_data(2, ' '):
    value = int(value)
    if direction == "forward":
        horizontal_loc += value
    elif direction == "down":
        depth_loc += value
    elif direction == "up":
        depth_loc -= value
horizontal_loc * depth_loc

1580000

In [3]:
horizontal_loc = 0
depth_loc = 0
aim = 0
for direction, value in get_daily_data(2, ' '):
    value = int(value)
    if direction == "forward":
        horizontal_loc += value
        depth_loc += value * aim
    elif direction == "down":
        aim += value
    elif direction == "up":
        aim -= value
horizontal_loc * depth_loc

1251263225

## Day 1

In [4]:
n_increases = 0
prev_x = 9e32
for value in get_daily_data(1):
        x = int(value)
        if x > prev_x:
            n_increases += 1
        prev_x = x
n_increases

1466

In [5]:
n_increases = 0
window = []
prev_sum = None
for value in get_daily_data(1):
    x = int(value)
    if len(window) < 3:
        window.append(x)
    if prev_sum is None:
        prev_sum = sum(window)
    if len(window) == 3:
        window.pop(0)
        window.append(x)
        new_sum = sum(window)
        if new_sum > prev_sum:
            n_increases += 1
        prev_sum = new_sum
n_increases

1491